<center>

<img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=300px/>

<h2>Python: исключения, менеджеры контекста</h2>
Сапожников Денис
<br />
<h4>2021</h4>

</center>

# Исключения

Часто в программах что-то идёт не так. Если ничего не предпринимать, они ломаются.

In [1]:
def parse_tskv(tskv: str) -> dict[str, int]:
    """Parse tskv string"""
    kvpairs = (keyvalue.split('=') for keyvalue in tskv.strip().split('\t'))
    return {k: int(v) for k, v in kvpairs}

log = [
    'banner_id=1\tshows=10\tclicks=1',
    'banner_id=2\tshows=15\tclicks=2',
    'banner_id=3\tshows=\tclicks=1',  # empty shows
]

for row in log:
    print(parse_tskv(row))

{'banner_id': 1, 'shows': 10, 'clicks': 1}
{'banner_id': 2, 'shows': 15, 'clicks': 2}


ValueError: invalid literal for int() with base 10: ''

Какие средства для обработки ошибок существуют?

- Специальные возвращаемые значения (Golang)
```go
i, err := strconv.Atoi("42")
if err != nil {
    fmt.Printf("couldn't convert number: %v\n", err)
    return
}
fmt.Println("Converted integer:", i)
```

- Исключения (Python)

In [42]:
int('abc')

ValueError: invalid literal for int() with base 10: 'abc'

- Исключения — специальный механизм языка для работы с ошибками.
- Прерывают нормальный ход исполнения программы.
- Сообщают о возникшей исключительной ситуации.
- Дают возможность обработать ошибку и восстановить работу программы.

Примеры исключений

In [46]:
[0] * int(1e16)

MemoryError: 

Примеры исключений

In [47]:
open('nonexistent.file')

FileNotFoundError: [Errno 2] No such file or directory: 'nonexistent.file'

Примеры исключений

In [48]:
[1, 2, 3] + 4

TypeError: can only concatenate list (not "int") to list


Примеры исключений

In [49]:
compile('a = 2 * 5 + 3)', '', 'exec')

SyntaxError: invalid syntax (<string>, line 1)

Иерархия встроенных исключений: https://docs.python.org/3/library/exceptions.html#exception-hierarchy

```
BaseException
 +-- SystemExit
 +-- KeyboardInterrupt
 +-- GeneratorExit
 +-- Exception
      +-- StopIteration
      +-- StopAsyncIteration
      +-- ArithmeticError
      |    +-- FloatingPointError
      |    +-- OverflowError
      |    +-- ZeroDivisionError
      +-- AssertionError
      +-- AttributeError
      +-- BufferError
      +-- EOFError
      +-- ImportError
      |    +-- ModuleNotFoundError
      +-- LookupError
      |    +-- IndexError
      |    +-- KeyError
      +-- MemoryError
      +-- NameError
      |    +-- UnboundLocalError
      +-- OSError
      |    +-- BlockingIOError
      |    +-- ChildProcessError
      |    +-- ConnectionError
      |    |    +-- BrokenPipeError
      |    |    +-- ConnectionAbortedError
      |    |    +-- ConnectionRefusedError
      |    |    +-- ConnectionResetError
      |    +-- FileExistsError
      |    +-- FileNotFoundError
      |    +-- InterruptedError
      |    +-- IsADirectoryError
      |    +-- NotADirectoryError
      |    +-- PermissionError
      |    +-- ProcessLookupError
      |    +-- TimeoutError
      +-- ReferenceError
      +-- RuntimeError
      |    +-- NotImplementedError
      |    +-- RecursionError
      +-- SyntaxError
      |    +-- IndentationError
      |         +-- TabError
      +-- SystemError
      +-- TypeError
      +-- ValueError
      |    +-- UnicodeError
      |         +-- UnicodeDecodeError
      |         +-- UnicodeEncodeError
      |         +-- UnicodeTranslateError
      +-- Warning
           +-- DeprecationWarning
           +-- PendingDeprecationWarning
           +-- RuntimeWarning
           +-- SyntaxWarning
           +-- UserWarning
           +-- FutureWarning
           +-- ImportWarning
           +-- UnicodeWarning
           +-- BytesWarning
           +-- ResourceWarning
```

Обработка исключений: `try...except`

In [57]:
filename = 'nonexistent.file'

try:
    fd = open(filename, 'r')
except FileNotFoundError:  # catch exceptions which satisfy isinstance(exc, FileNotFoundError)
    print(f'File {filename!r} does not exist')

File 'nonexistent.file' does not exist


Обработка исключений: `try...except...except`

In [1]:
filename = 'nonexistent.file'

try:
    fd = open(filename, 'r')
except Exception as e:  # the first matching except clause is triggered
    print(f'Exception occured while reading file {filename!r}: {e!r}')
except FileNotFoundError:
    print(f'File {filename!r} does not exist')
except (TypeError, ValueError, MemoryError) as e:
    print('Just to demonstrate a tuple of exceptions')

File 'nonexistent.file' does not exist


Обработка исключений: `try...except...else...finally`

In [4]:
try:
    f = open("fenr.ipynb", 'r') # something dangerous
except Exception as e:  # scope failure
    print(f'Something bad happened: {e!r}')
else:  # scope success
    print('Nothing bad happened')
finally:  # scope exit
    f.close()
    print('Print this no matter what')

Something bad happened: FileNotFoundError(2, 'No such file or directory')
Print this no matter what


Стратегии обработки ошибок: **Look Before You Leap**

In [79]:
def ctr(shows, clicks):
    """Returns banner click-through rate"""
    if shows == 0:
        return 0
    return clicks / shows

Стратегии обработки ошибок: **It's easier to ask for forgiveness than permission**

In [80]:
def ctr(shows, clicks):
    """Returns banner click-through rate"""
    try:
        return clicks / shows
    except ZeroDivisionError:
        return 0

<div class="alert alert-danger">
<b>Антипаттерн: </b> Ловить BaseException
</div>

In [ ]:
try:
    do_dangerous()
except:  # catch everything, even KeyboardInterrupt
    pass

In [ ]:
try:
    do_dangerous()
except BaseException:  # same as above
    pass

Старайтесь максимально конкретизировать исключения в except

Бросить исключение можно с помощью ключевого слова `raise`

In [72]:
raise ValueError('Positive integer expected')

ValueError: Positive integer expected

Исключение должно быть объектом типа BaseException или его наследника

In [73]:
raise 42

TypeError: exceptions must derive from BaseException

`raise` без аргумента перебрасывает последнее пойманное исключение.

In [3]:
try:
    raise RuntimeError('Crash hard')
except:
    print('Unknown error occured, no chance to recover, run!')
    raise

Unknown error occured, no chance to recover, run!


RuntimeError: Crash hard

In [1]:
raise

RuntimeError: No active exception to reraise

`raise ... from ...`

In [84]:
def ctr(shows, clicks):
    """Returns banner click-through rate"""
    try:
        return clicks / shows
    except ZeroDivisionError as e:
        raise ValueError('Bad banner') from e
ctr(0, 1)

ValueError: Bad banner

Цепочки исключений

In [2]:
try:
    raise ValueError('Bad value')
except ValueError:
    raise RuntimeError('Dunno what to do!')

RuntimeError: Dunno what to do!

Можно создавать свои классы исключений, достаточно отнаследоваться от `Exception`. Хорошая практика — наследовать свои исключения от общего предка, чтобы их было удобнее ловить. Пример кастомных исключений: https://github.com/psf/requests/blob/master/requests/exceptions.py

In [10]:
class ShoeError(Exception):
    pass

class WrongFootError(ShoeError):
    def __str__(self):
        return f'Try another one!'
        
raise WrongFootError([1, 2, 3])

WrongFootError: Try another one! ([1, 2, 3],)

Как устроены объекты-исключения

In [86]:
try:
    raise ValueError(1, 2, 3)
except Exception as e:
    exc = e

In [87]:
exc.args  # аргументы конструктора

(1, 2, 3)

In [88]:
exc.__cause__  # причина исключения, устанавливается при raise EXC from CAUSE
exc.__context__  # последнее пойманное исключение, для цепочек исключений
exc.__traceback__

In [ ]:
exc.with_traceback(tb)  # устанавливает __traceback__ в новое значение tb

### Warnings

In [4]:
import numpy as np

np.int32(1) / np.int32(0)

/tmp/ipykernel_1371/3395287266.py:3: RuntimeWarning: divide by zero encountered in int_scalars
  np.int32(1) / np.int32(0)


inf

In [1]:
import numpy as np

try:
    np.int32(1) / np.int32(0)
except Exeption:
    print("Exception")

/tmp/ipykernel_1543/2178758311.py:4: RuntimeWarning: divide by zero encountered in int_scalars
  np.int32(1) / np.int32(0)


То есть не смотря на то, что Warnings - наследник Exeption, всё равно не удается поймать и обработать warning.

### Warnings

In [6]:
import numpy as np
import warnings

warnings.filterwarnings("error")
try:
    np.int32(1) / np.int32(0)
except Exception as e:
    print(f"Exception {e!r}")

warnings.resetwarnings()

Exception RuntimeWarning('divide by zero encountered in int_scalars')


In [7]:
import numpy as np
import warnings

warnings.filterwarnings("ignore")
try:
    np.int32(1) / np.int32(0)
except Exception as e:
    print(f"Exception {e!r}")

warnings.resetwarnings()

Полезные штуки

- `sys.exc_info()` — возвращает информацию о текущем обрабатываемом исключении
- Модуль `traceback`
- Модуль `warnings`

# Менеджеры контекста

Начнём издалека.

Доклад Скотта Майерса "Why C++ Sails When the Vasa Sank" в Яндексе, 2014.

__"What you would consider the single most important feature in C++?"__

https://youtu.be/ltCgzYcpFUI?t=952

Как гарантировать, что некоторое действие будет выполнено вне зависимости от того, произошло исключение или нет?

In [96]:
def do_something_dangerous(fd):
    raise RuntimeError('Not today!')

fd = open('myfile.txt', 'w')
try:
    do_something_dangerous(fd)
finally:
    print('Closing file')
    fd.close()
    print('File closed')

Closing file
File closed


RuntimeError: Not today!

Менеджеры контекста предоставляют удобный способ провести инициализацию и гарантированную финализацию "контекста".

In [ ]:
r = aquire_resource()
try:
    use_resource(r)
finally:
    release_resource(r)

In [ ]:
with aquire_resource() as r:
    use_resource(r)

Примеры менеджеров контекста: `open`

In [3]:
with open('filename.txt', 'w') as fd:
    fd.write("Hello")
# file is closed
fd.write("world")

ValueError: I/O operation on closed file.

Примеры менеджеров контекста: `tempfile`

In [ ]:
import tempfile

with tempfile.TemporaryFile() as tmp:
    do_something(tmp)
# tmp file is removed

Примеры менеджеров контекста: Python Database API

In [ ]:
import psycopg2
with psycopg2.connect(...) as conn:
    with conn.cursor() as cursor:
        cursor.execute('SELECT * FROM MyTable', params)
        result = cursor.fetchall()
# cursor.close() is called
# conn.commit() or conn.rollback() is called

Примеры менеджеров контекста: `pytest`

In [9]:
import pytest
with pytest.raises(ZeroDivisionError):
    a = 1 / 0
# ZeroDivisionError is not expected to occur anymore and will cause test to fail
with pytest.raises(ZeroDivisionError):
    a = 0 / 1

Failed: DID NOT RAISE <class 'ZeroDivisionError'>

Примеры менеджеров контекста: `warnings`

In [15]:
import numpy as np
import warnings

with warnings.catch_warnings(record=True) as w:
    # Cause all warnings to always be triggered.
    warnings.simplefilter("always")
    np.int32(1) / np.int32(0)
    np.log(0)
    
    for warn in w:
        print(warn)

{message : RuntimeWarning('divide by zero encountered in int_scalars'), category : 'RuntimeWarning', filename : '/tmp/ipykernel_1477/105942860.py', lineno : 7, line : None}
{message : RuntimeWarning('divide by zero encountered in log'), category : 'RuntimeWarning', filename : '/tmp/ipykernel_1477/105942860.py', lineno : 8, line : None}


Синтаксис выражения `with`

In [ ]:
# nested contexts
with first() as f, second as s():
    do_something(f, s)

In [ ]:
# same as above
with first() as f:
    with second as s():
        do_something(f, s)

In [ ]:
with third():  # <as NAME> part as optional
    do_something()

Менеджеры контекста — объекты, реализующие специальный протокол

In [100]:
import traceback

class MyContextManager:
    def __enter__(self):
        ...  # initialize context
        return context
    
    def __exit__(self, exc_type: type, exc_value: BaseException, traceback: traceback):
        ...  # finalize context
        if exc_value is not None:
            return True  # return True from __exit__ to suppress the exception

Семантика

In [ ]:
with acquire_resource() as resource:
    use_resource(resource)

In [ ]:
manager = acquire_resource()
resource = manager.__enter__()
try:
    use_resource(resource)
finally:
    exc_type, exc_value, traceback = sys.exc_info()
    suppress = manager.__exit__(exc_type, exc_value, traceback)
    if exc_value is not None and not suppress:
        raise exc_value

Полушуточный пример

In [93]:
class Tag:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        print('<{}>'.format(self.name))
    def __exit__(self, *args):
        print('</{}>'.format(self.name))

        
with Tag('table'):
    with Tag('tr'):
        with Tag('td'):
            print('cell 1')
        with Tag('td'):
            print('cell 2')

<table>
<tr>
<td>
cell 1
</td>
<td>
cell 2
</td>
</tr>
</table>


`contextlib.contextmanager` — удобный способ создавать менеджеры контекста

In [102]:
from contextlib import contextmanager

@contextmanager
def mycm():
    print('before')
    yield 42  # yep, it is a generator
    print('after')
    
with mycm() as r:
    print(f'got {r}')
    
with mycm() as r:
    raise RuntimeError('Oops')
# 'after' is not printed!

before
got 42
after
before


RuntimeError: Oops

Но работать с `contextlib.contextmanager` надо аккуратно

In [103]:
from contextlib import contextmanager

@contextmanager
def mycm():
    print('before')
    try:
        yield 42
    finally:
        print('after')

with mycm() as r:
    raise RuntimeError('Oops')

before
after


RuntimeError: Oops

В модуле `contextlib` есть и другие полезные штуки:
- `contextlib.ContextDecorator` — базовый класс для менеджеров контекста, их потом можно будет использовать как декораторы для функций
- `contextlib.ExitStack` — позволяет использовать неизвестное заранее количество "ресурсов", динамически управлять менеджерами контекста
- См. документацию